In [10]:
import yfinance as yf
import datetime

In [12]:
stock = yf.download('BTC-USD', period='1d', interval='1m')
stock

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-09 00:00:00+00:00,56704.597656,56704.597656,56704.597656,56704.597656,56704.597656,0
2024-07-09 00:02:00+00:00,56666.656250,56666.656250,56666.656250,56666.656250,56666.656250,0
2024-07-09 00:03:00+00:00,56682.386719,56682.386719,56682.386719,56682.386719,56682.386719,0
2024-07-09 00:05:00+00:00,56650.324219,56650.324219,56650.324219,56650.324219,56650.324219,0
2024-07-09 00:08:00+00:00,56706.832031,56706.832031,56706.832031,56706.832031,56706.832031,0
...,...,...,...,...,...,...
2024-07-09 13:40:00+00:00,57295.679688,57295.679688,57295.679688,57295.679688,57295.679688,0
2024-07-09 13:41:00+00:00,57213.566406,57213.566406,57213.566406,57213.566406,57213.566406,0
2024-07-09 13:44:00+00:00,57082.761719,57082.761719,57082.761719,57082.761719,57082.761719,0


In [13]:
stock.index

DatetimeIndex(['2024-07-09 00:00:00+00:00', '2024-07-09 00:02:00+00:00',
               '2024-07-09 00:03:00+00:00', '2024-07-09 00:05:00+00:00',
               '2024-07-09 00:08:00+00:00', '2024-07-09 00:09:00+00:00',
               '2024-07-09 00:10:00+00:00', '2024-07-09 00:12:00+00:00',
               '2024-07-09 00:14:00+00:00', '2024-07-09 00:15:00+00:00',
               ...
               '2024-07-09 13:32:00+00:00', '2024-07-09 13:33:00+00:00',
               '2024-07-09 13:36:00+00:00', '2024-07-09 13:38:00+00:00',
               '2024-07-09 13:39:00+00:00', '2024-07-09 13:40:00+00:00',
               '2024-07-09 13:41:00+00:00', '2024-07-09 13:44:00+00:00',
               '2024-07-09 13:45:00+00:00', '2024-07-09 13:47:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datetime', length=533, freq=None)

In [15]:
stock.tz_convert('CET')

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-09 02:00:00+02:00,56704.597656,56704.597656,56704.597656,56704.597656,56704.597656,0
2024-07-09 02:02:00+02:00,56666.656250,56666.656250,56666.656250,56666.656250,56666.656250,0
2024-07-09 02:03:00+02:00,56682.386719,56682.386719,56682.386719,56682.386719,56682.386719,0
2024-07-09 02:05:00+02:00,56650.324219,56650.324219,56650.324219,56650.324219,56650.324219,0
2024-07-09 02:08:00+02:00,56706.832031,56706.832031,56706.832031,56706.832031,56706.832031,0
...,...,...,...,...,...,...
2024-07-09 15:40:00+02:00,57295.679688,57295.679688,57295.679688,57295.679688,57295.679688,0
2024-07-09 15:41:00+02:00,57213.566406,57213.566406,57213.566406,57213.566406,57213.566406,0
2024-07-09 15:44:00+02:00,57082.761719,57082.761719,57082.761719,57082.761719,57082.761719,0
